<a href="https://colab.research.google.com/github/Shahriar10k/RAG-with-NSU/blob/main/Title_Text_Dataset_499A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping North South University website

imports

In [ ]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
import bs4
from html.parser import HTMLParser
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import numpy as np
import pandas as pd
import time

python's HTMLParser

In [ ]:
from html.entities import name2codepoint

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)
        for attr in attrs:
            print("     attr:", attr)

    def handle_endtag(self, tag):
        print("End tag  :", tag)

    def handle_data(self, data):
        print("Data     :", data)

    def handle_comment(self, data):
        print("Comment  :", data)

    def handle_entityref(self, name):
        c = chr(name2codepoint[name])
        print("Named ent:", c)

    def handle_charref(self, name):
        if name.startswith('x'):
            c = chr(int(name[1:], 16))
        else:
            c = chr(int(name))
        print("Num ent  :", c)

    def handle_decl(self, data):
        print("Decl     :", data)

parser = MyHTMLParser()

Demo run Using BS4 

In [ ]:
document = requests.get('http://www.northsouth.edu/about/brief-history.html')
soup = BeautifulSoup(document.text, "html.parser")
print(soup.title.string)
#print(soup.find('h1').text.strip())
print(soup.h2.string)
p = (soup.find_all('p'))

for i in p:
  print(i.string)

divs = soup.find_all('div')

for div in divs:
  try:
    if len(div.string) > 0:
      print(div.string)
  except:
    continue

NSU: A BRIEF HISTORY | North South University
NSU: A BRIEF HISTORY
North South University (NSU), the first private university in Bangladesh, was established in 1992 by the then Foundation for Promotion of Education and Research (FPER), a charitable, non-profit, non-commercial and non-political organization. The FPER later was renamed as the NSU Foundation and is presently called The North South Foundation for Education and Research. The Foundation is comprised of a group of eminent industrialists, prominent patrons of education, notable philanthropists, widely experienced academics and senior civil servants of the country. In the early 1990s, they had a dream to set up a world-class university as a center of excellence in higher education in the private sector. Their dedication, tireless efforts, and hard work paved the way for the approval of the establishment of NSU.
Since, at that time, there was no relevant law in the country to set up and operate a university in the private sector

Fetch all links from the base index page (Does not contain sub-page links)

In [ ]:
index = 'http://www.northsouth.edu/'
document = requests.get(index)
links = []
soup = BeautifulSoup(document.text, "html.parser")

for link in soup.findAll('a'):
  var = link.get('href')
  if(var == '#' or var is None):
    continue
  else:
    if('http' in var):
      #print(var)
      links.append(var)
    else:
      var = index + var
      links.append(var)
      #print(var)


Cleaning, remove PDF links, career, archive

In [ ]:
broken_or_denied_links = [] # hold

def format(links):
  prev_length = len(links)
  new_length = 0

  while(prev_length != new_length):
    prev_length = new_length
    for link in links:
      if('pdf' in link):
        links.remove(link)
      elif 'journal' in link:
        links.remove(link)
      elif ('career' in link):
        links.remove(link)
      elif 'archive' in link:
        links.remove(link)
      elif 'youtube' in link:
        links.remove(link)
      elif 'alumni' in link:
        links.remove(link)
      elif 'drive.google' in link:
        links.remove(link)
      elif('.instagram.' in link or 'google' in link or 'twitter' in link or 'facebook' in link):
        links.remove(link)
      elif 'rds3' in link or 'login' in link:
        links.remove(link)
      else:
        pass
    new_length = len(links)



def format_major(links):
  prev_length = len(links)
  new_length = 0

  while(prev_length != new_length):
    prev_length = new_length
    for link in links:
      if'pdf' in link or '#' in link:
        links.remove(link)
      elif 'cpc' in link or 'gmail' in link or 'opac' in link or '.jpg' in link:
        links.remove(link)
      elif 'journal' in link or 'newsletter' in link or 'news-events' in link:
        links.remove(link)
      elif ('career' in link or 'contact-us' in link or '/upcoming-events/' in link):
        links.remove(link)
      elif 'archive' in link or 'nsustartup' in link or 'startup' in link:
        links.remove(link)
      elif 'youtube' in link or 'you' in link:
        links.remove(link)
      elif 'alumni' in link or 'zoom' in link or 'docx' in link:
        links.remove(link)
      elif 'drive.google' in link or 'announcements' in link or 'research-committee' in link:
        links.remove(link)
      elif('.instagram.' in link or 'google' in link or 'twitter' in link or 'facebook' in link):
        links.remove(link)
      elif 'rds3' in link or 'login' in link:
        links.remove(link)
      else:
        pass
    new_length = len(links)


format(links)

print(len(links))
#for link in links:
#  print((link))

108


Total formatted links: 

In [ ]:
len(links)

108

fetching links from the list of links (links from sub pages) | 4 mins execution time

In [ ]:
links_major = [] 

total = len(links)
i = 1
for link in links:
  index = link
  document = requests.get(index)
  soup = BeautifulSoup(document.text, "html.parser")
  if i%10==0:
    print("fetching links:  %d of 133" %i)
  i+=1
  for l in soup.findAll('a'):
    var = l.get('href')
    if(var == '#' or var is None):
      continue
    else:
      if('http' in var):
        #print(var)
        links_major.append(var)
      else:
        var = "www.northsouth.edu/" + var
        links_major.append(var)
        #print(var)

print("***  Done.  ***")
links_major += links # copy contents

fetching links:  10 of 133
fetching links:  20 of 133
fetching links:  30 of 133
fetching links:  40 of 133
fetching links:  50 of 133
fetching links:  60 of 133
fetching links:  70 of 133
fetching links:  80 of 133
fetching links:  90 of 133
fetching links:  100 of 133
***  Done.  ***


remove duplicates and format

In [ ]:
links_major = list(dict.fromkeys(links_major))

format_major(links_major)
#format(links_major)

print(len(links_major))

1398


In [ ]:
title_list = []
text_list = []

def fetch(link):
  content = requests.get(link)
  #text_file = open("data.txt", "a")
  title_df = ""
  document = ""
  soup = BeautifulSoup(content.text, "html.parser")

  try: #check
    title = soup.find('title').string
    #text_file.write(str(title))
    title_df = title
    print(title)
  except:
    print(f"*Title does not exist for this page. \n * {link}")

  try:
    for var in soup.findAll('h1'):
      #text_file.write(str(var.text.strip()))
      print(var.string)
      #document += (var.string)

    for var in soup.findAll('h2'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      #document += str(var.text.strip())

    for var in soup.findAll('h3'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      #document += str(var.text.strip())

    for var in soup.findAll('h6'):
      #text_file.write(str(var.text.strip()))
      print(var.text.strip())
      #document += str(var.text.strip())
  except:
    print(f"error in text extraction for h elements \n {link}")
    #broken_or_denied_links.append(link)

  try:
    for var in soup.findAll('p'):
      #text_file.write(str(var.text.strip()))
      print(var.string)
      document += str(var.text)
  except:
      print(f"*error in text extraction* \n {link}")
      broken_or_denied_links.append(link)

  divs = soup.find_all('div')

  for div in divs:
    try:
      if len(div.string) > 0:
        print(div.string)
        document += div.string
    except:
      continue

  title_list.append(title_df)
  text_list.append(document)
  #text_file.close()

Filter out non-nsu websites and mail links

In [ ]:
link_backup = links_major #copy just in case

In [ ]:
print(f"links before: {len(links_major)}")

for link in links_major:
  if not ("nsu" in link or "northsouth" in link):
    links_major.remove(link)
  elif "mailto" in link or 'dept-notices/' in link or 'research-office/' in link or 'cps' in link or 'edu/?' in link or 'mppg' in link:
    links_major.remove(link)
  elif 'xls' in link or 'png' in link or 'staff' in link or 'javascript' in link or 'canvas' in link:
    links_major.remove(link)

print(f"links after: {len(links_major)}")

links before: 739
links after: 739


In [ ]:
c=0
for i in links_major:
  if 'staff' in i:
    c+=1
print(f"links after: {c}")

links after: 21


In [ ]:
# THIS CELL IS USED FOR DEBUG
#format_major(links_major)
#print(f"links after: {len(links_major)}")

#for i in links_major:
#  print(i)

fetch contents from links

In [ ]:
#dataframe = pd.DataFrame(columns=['Title', 'Text'])
dataframe = pd.DataFrame(columns=['Title', 'Text'])

i=1
for link in links_major:
  try:
    print(f"Fetching {i} of 1643")
    i+=1
    if i%100==0:
      time.sleep(10)
      print("\n \n ** Waiting **")
    fetch(link)
  except:
    print(f"error at{link}")
    print(f"\n \n ** Exception_no_host ** {link}")
    broken_or_denied_links.append(link)
    #continue

Streaming output truncated to the last 5000 lines.
None
None
None
None
 
 
 
 
 
 
 
Fetching 328 of 1643
Medical - Department of Electrical and Computer Engineering
Electrical & Computer Engineering
North South University
Medical
North South University
Resources
Students at NSU can enjoy their healthcare facility through NSU Medical Center. The Healthcare Center is conveniently located at the 4th level of South Academic Building (SAC).
For details about the student healthcare support, please follow the link below
NSU Medical Center
 
None
None
 
 
 
 
 
 
 
Fetching 329 of 1643
Industry - Department of Electrical and Computer Engineering
Electrical & Computer Engineering
North South University
Industry
North South University
Industry
None
None
None
None
 
 
 
 
 
 
 
Fetching 330 of 1643
Advisory Board - Department of Electrical and Computer Engineering
Electrical & Computer Engineering
North South University
Advisory Board
North South University
Purpose:

Industry
The Department of E

# 2nd approach, attempt to auto fetch titles

In [ ]:
# SAVE Broken links? not all are broken it seems. 
broken_links = pd.DataFrame(broken_or_denied_links)
broken_links.to_csv(r'broken_links', index=False)

print(len(broken_or_denied_links))
print(len(links_major))
print(len(links))

base_links = pd.DataFrame(links)
base_links.to_csv(r'base_links', index=False)

514
739
108


In [ ]:
count = 0
count2 = 0
link_2 = links_major

for i in broken_or_denied_links:
  if i in links_major:
    links_major.remove(i) 
    count+=1

for i in links:
  if i in links_major:
    links_major 
    count2+=1

#print(len(broken_or_denied_links))
print(count)
print(count2)



In [ ]:
print(len(links_major))

225


In [ ]:
prev_broken_list = broken_or_denied_links
broken_or_denied_links = []

Lets try wtih the new approach

In [ ]:
title_list = []
text_list = []

def fetch(link):
  content = requests.get(link)
  title_df = ""
  document = ""
  soup = BeautifulSoup(content.text, "html.parser")

  try:
    title = soup.find(class_ ='title').string
    title_df = title
  except:
    try:
      title_df = soup.find('title').string
    except:
      title_df = ("Title not found in HTML. ")

  try:
    for var in soup.findAll('p'):
      #print(var.string)
      document += str(var.text) + " "
  except:
      #print(f"*error in text extraction* \n {link}")
      #broken_or_denied_links.append(link)

      pass

  divs = soup.find_all('div')

  for div in divs:
    try:
      if len(div.string) > 0:
        #print(div.string)
        document += div.string
    except:
      continue

  title_list.append(title_df)
  text_list.append(document)
  #text_file.close()

In [ ]:
format_major(links)
print(len(links))

format_major(links_major)
print(len(links_major))

links_major += links

links_major = list(dict.fromkeys(links_major))
format_major(links_major)

print(len(links_major))


i=0

#small_links = ["http://www.northsouth.edu/about/brief-history.html", "http://www.northsouth.edu/about/vms.html"]
#small_links = ["http://www.northsouth.edu/about/facts.html"]
#^used for debug

length = len(links_major)
for link in links_major:
  i+=1
  if i % 5 == 0:
    print(f"Links left to scrape: {length - i}")
  fetch(link)

print("  Done. ")

108
225
319
Links left to scrape: 314
Links left to scrape: 309
Links left to scrape: 304
Links left to scrape: 299
Links left to scrape: 294
Links left to scrape: 289
Links left to scrape: 284
Links left to scrape: 279
Links left to scrape: 274
Links left to scrape: 269
Links left to scrape: 264
Links left to scrape: 259
Links left to scrape: 254
Links left to scrape: 249
Links left to scrape: 244
Links left to scrape: 239
Links left to scrape: 234
Links left to scrape: 229
Links left to scrape: 224
Links left to scrape: 219
Links left to scrape: 214
Links left to scrape: 209
Links left to scrape: 204
Links left to scrape: 199
Links left to scrape: 194
Links left to scrape: 189
Links left to scrape: 184
Links left to scrape: 179
Links left to scrape: 174
Links left to scrape: 169
Links left to scrape: 164
Links left to scrape: 159
Links left to scrape: 154
Links left to scrape: 149
Links left to scrape: 144
Links left to scrape: 139
Links left to scrape: 134
Links left to scrape: 129


In [ ]:
dataframe = pd.DataFrame(columns=['Title', 'Text'])
title = pd.DataFrame(title_list, columns=["Title"])
text = pd.DataFrame(text_list, columns=["Text"])

temp = pd.concat([title, text], axis = 1)

dataframe = pd.concat([dataframe, temp], axis = 0)

In [ ]:
dataframe.to_csv(r'data.csv', index=False)

In [ ]:


print(len(title_list))
print(len(text_list))

544
544


In [ ]:
#for i in links:
#  print(i)



# Preparing the dataset

In [ ]:
#title = pd.DataFrame(title_list, columns=["Title"])
#text = pd.DataFrame(text_list, columns=["Text"])

#temp = pd.concat([title, text], axis = 1)

#dataframe = pd.concat([dataframe, text], axis = 0)

In [ ]:
#print(dataframe.head())

  Title                                               Text
0   NaN                                                   
1   NaN  Total 2,178 title books collection are availab...
2   NaN  \n                            North South Univ...
3   NaN  Most Visited LinksUndergraduateGraduateWelcome...
4   NaN                                                   


In [ ]:
print(len(title_list))
print(len(text_list))

544
544


In [ ]:
dataframe.dropna(inplace=True)
dataframe.drop_duplicates(inplace=True)
dataframe.to_csv(r'dataframe.csv', index=False)

remove blank strings from list

In [ ]:
print(len(text_list))
# remove
while("" in text_list) :
    text_list.remove("")

print(len(text_list))

540
540


sep

In [ ]:
separated_doc = []
separated_title = []
# txt is a dummy list for debug :) , 0-th index consists of 166 words.
#txt = ['But I must explain to you how all this mistaken idea of denouncing pleasure and praising pain was born and I will give you a complete account of the system, and expound the actual teachings of the great explorer of the truth, the master-builder of human happiness. No one rejects, dislikes, or avoids pleasure itself, because it is pleasure, but because those who do not know how to pursue pleasure rationally encounter consequences that are extremely painful. Nor again is there anyone who loves or pursues or desires to obtain pain of itself, because it is pain, but because occasionally circumstances occur in which toil and pain can procure him some great pleasure. To take a trivial example, which of us ever undertakes laborious physical exercise, except to obtain some advantage from it? But who has any right to find fault with a man who chooses to enjoy a pleasure that has no annoying consequences, or one who avoids a pain that produces no resultant pleasure', 'abc', " ", "", "   "]
print("before splitting texts: %d" %(len(text_list)))
print("before splitting titles: %d" %(len(title_list)))


iterator = 0
for s in text_list:
  title_at_i = title_list[iterator]
  iterator += 1
  counter = 0
  words = s.split()
  for i in range(0, len(words), 100):
    chunked_words = words[i : i+100]
    recombined_words = " ".join(chunked_words)
    recombined_words = recombined_words.strip() # strip trailing white spaces
    #separated_doc.append(recombined_words)
    if len(words) == 0:
      counter+=1
      continue
    else:
      separated_doc.append(recombined_words)
      separated_title.append(title_at_i)

print("after splitting: Title: %d" %(len(separated_title)))
print("after splitting: Text: %d" %(len(separated_doc)))
print("text with blank spaces in the last: %d" %(counter))

before splitting texts: 540
before splitting titles: 544
after splitting: Title: 2874
after splitting: Text: 2874
text with blank spaces in the last: 0


In [ ]:
# No need to run this cell, used for debugging purposes.
for s in separated_doc:
  print(len(s.split()))

In [ ]:
#title = pd.DataFrame(separated_title, columns=["Title"])
text = pd.DataFrame(list(zip(separated_title, separated_doc)), columns=["Title", "Text"])

#temp = pd.concat([title, text], axis = 1)
dataframe = dataframe.iloc[0:0]
dataframe = pd.concat([dataframe, text], axis = 0)

In [ ]:
print(dataframe.head())
#dataframe.dropna(inplace=True)
dataframe.drop_duplicates(inplace=True)
dataframe.to_csv(r'dataframe_new.csv', index=True)

                            Title  \
0              Home - NSU Library   
1              Home - NSU Library   
2  Research Support - NSU Library   
3  Research Support - NSU Library   
4   Upcoming Events – NSU Library   

                                                Text  
0  Total 2,178 title books collection are availab...  
1  Minister of Information and Broadcasting Minis...  
2  Overview The NSU Library is dedicatedly playin...  
3  research output management and analysis, subje...  
4  Developed & Maintained by NSU Library IT Team....  
